## Before running any of these commands, make sure you have AWS CLI __[installed](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#getting-started-install-instructions)__ and __[configured](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-quickstart.html)__ 

### Get User List

In [ ]:
import boto3, csv

iam = boto3.client('iam')
user_list = []
dict_index = 0
response = iam.list_users()
for user in response["Users"]:
    user_list.append(user["UserName"])

print(user_list)

### Get Policy List

In [ ]:
policy_list = []
list_index = 0
for user in user_list:
    response = iam.list_attached_user_policies(UserName=user)
    if(len(response["AttachedPolicies"]) != 0):
        policy_list.append(response["AttachedPolicies"][0]["PolicyName"])
    else:
        policy_list.append(response["AttachedPolicies"])
    list_index += 1

print(policy_list) 

### Get User Groups

In [ ]:
groups_list = []
list_index = 0
for user in user_list:
    user_groups = []
    response = iam.list_groups_for_user(UserName=user)
    for group in response['Groups']:
        user_groups.append(group['GroupName'])
    groups_list.append(user_groups)

print(groups_list)

### Format List for Export

In [ ]:
new_list = []
list_index = 0

for user in user_list:
    new_list.append(f'{user},{policy_list[list_index]},{groups_list[list_index]}')
    list_index += 1


def replaceStuff(new_list, replacement):
    new_new_list = []
    for each in new_list:
        new_str = each.replace(replacement, '')
        new_new_list.append(new_str)
    return new_new_list

new_list = replaceStuff(new_list, '\'')
new_list = replaceStuff(new_list, '[')
new_list = replaceStuff(new_list, ']')
print(new_list)

# Export as CSV

In [ ]:
with open('user_list.csv', 'w') as csvoutput:
    titlerow = "Users,Individual Policies,Groups\n"
    csvoutput.write(titlerow)
    for each in new_list:
        csvoutput.write(f'{each}\n')